### Original Codes

In [88]:
def left(i):
    return 2*(i) + 1 #as index starts from 0, actually the left will be the odd number

def right(i):
    return 2*(i+1) #as the index starts from 0, the actual right will be even

def heapify1(heap, i, t):
    """
    Parameters
    ----------
    heap : list of floats
        Assume that the heap size is the length of the heap
        
    Note
    ----
    No output is needed. This function should modify (if necessary) 
    heap in-place.
    
    """
    #i starts from 0
    heap_size = len(heap)-1 #as we assumed that the heap size = length of the heap above, but index number cannot go over heapsize-1 as index starts from 0
    L = left(i) 
    R = right(i)
    
    if L <= heap_size and heap[L][t]>heap[i][t]:
        largest = L
    else:
        largest = i
        
    if R <= heap_size and heap[R][t] > heap[largest][t]:
        largest = R
        
    if largest != i:
        heap[i], heap[largest] = heap[largest], heap[i] #exchange the numbers
        heapify1(heap, largest, t)

def build_max_heap_list(A,t):
    """
    Parameters
    ----------
    A : a list of floats
    
    Note
    ----
    No output is needed. The function should turn A into 
    a valid max heap, in-place.
    
    After building a max heap, do a heap sort
    
    """
    heap_size = len(A)-1
    for i in range(heap_size//2,-1,-1): #range from 0 to (floor of heap_size/2)-1
        heapify1(A,i,t)
        
def heap_sort(A,t):
    build_max_heap_list(A,t)
    for i in range(len(A)):
        B = A[i:]
        build_max_heap_list(B,1)
        A[i:] = B
    return A

from itertools import chain

### New Codes

In [112]:
#codes and comments for HCs and LOs on https://github.com/Jeongwoo-KGI/Optimal-Task-Schedule-Generator
class todo(object):    
    """
    This class builds each to do works independently.
    
    Attributes
    ---------
    ids : int
        It represents the id of that to-do. Assigned accordingly to input sequence, which means id does not matter that much
    
    description : str
        representing what to do
    
    duration : int
        representing the estimated duration in min.
    
    dependencies: list
        representing what needs to be done before execution (if no dependencies, set to 0)
        list as an input, meaning that I want to do this before the 'dependencies' is done represented by ids
    
    multi_tasking : int
        representing if it is multitasking possible work(1) or not(0)
        int as an input
        
    urgency : int
        d-day left for this thing to be done
        if d-day does not exist, enter -1
        
    importance : int
        0 : not important and trivial
        1 : not important in daily lives, but important in the far future (more than 5 years)
        2 : some what important in daily lives, as it will become important in near future (after 6 months ~ 4 years)
        3 : important to do in daily life, and it still will be in the future
        4 : whatever it is, it is very important for it to be done by the d-day(urgency)
        
    value : calculated from urgency and importance
        calculates the value of the work/do-do to be done, higher scores will be pre-assigned
        As a task with smaller d-day is more urgent thing to be done, the function goes like this.
        value = importance*10/(urgency+1) (if urgency >= 0)
        if it is not urgent, it will be equivalent to the urgency(d-day) set to 30
        
    time : float
        from 0 to 24, it represents the hour when the work is being done.
        automatically set to 0 in the first place before sorting
        this is here to be used by class 'make_a_todo_list'
    -------------
    self.task: list
        a list that holds all objects made from this class
    """
    
    def __init__(self, ids = 0, description = '', duration ='', dependencies = [], multi_tasking =0, urgency = 0, importance = 0, time = 0):
        self.ids = ids #intobject
        self.description = description #str object
        self.duration = duration #float object, in min(ex, 1hr 30 min = 90)
        self.dependencies = dependencies#task's ids to be done before the activity
        self.multi_tasking = multi_tasking #int = multitasking possible with that work id, empty list= multitasking impossible
        if urgency >= 0 : #how important is this work? Is it urgent? Is it important but not urgent?
            self.value = (importance*10)/(urgency+1)
        else: #base rate of the importance will be d-30
            self.value = (1/31)*(importance*10) 
        self.time = time #means that it is not yet set when to do the work, time refers to the hour when to do the work(ex, time = 13 means the work will start from 1pm)
        
    def aslist(self):
        #return the todo list
        return [self.ids, self.description, self.duration, self.dependencies, self.multi_tasking, self.value, self.time, 0]
        
    
class todo_list(object): #make a list of given classes
    """
    After creating objects from class todo, create the list what task should I do to make my day efficient
    tasks is the list of objects created by class todo
    wakeup_hr is there to set the time for waking up. This can be changed by wakeup_change method
    """
    
    def __init__(self, tasks, wakeup_hr = 8):
        self.tasks = tasks
        #wake up has a value of 100, because if we don't wake up, there will be no start of the day
        self.wake_up = [0, 'wakeup', 0, [], 0, 100, wakeup_hr] 
        self.hr = [[self.wake_up[6], self.wake_up[6], self.wake_up]]
        self.final_sort = []
        
    def wakeup_change(self):
        wakeup_time = int(input("When do you want to start your day tomorrow?"))
        self.wake_up = [1,'wakeup', 0, [], 0, 100, wakeup_time]
        self.hr =  [[self.wake_up[6], self.wake_up[6], self.wake_up]]
    
    def sorting(self): #topological sorting reference from cell 3
        #initial setups
        sorted_by_value = heap_sort(self.tasks, 5) #first sort them by values thus it is likely for higher value to start first by using a queue structure to sort all tasks
        queue = []
        length = len(sorted_by_value)
        indegrees = [0]*length
        arrows = []
        
        #maping nodes
        mapping = {} #mapping [task to start == sort_by_value[:][3] individually, after tasks == sort_by_value[:][0] individually]
        for i in range(len(sorted_by_value)):
            if sorted_by_value[i][3] == []:
                mapping[sorted_by_value[i][0]] = []
            else:
                mapping[sorted_by_value[i][0]] = sorted_by_value[i][3]
                arrows.append(sorted_by_value[i][3])
        arrows = list(chain.from_iterable(arrows))
        
        #make indegrees: number of arrows heading towards the ids = 'index number of indegrees' + 1
        for i in arrows:
            indegrees[i-1] += 1

        #sorte prima: pick the ones to start, it should have 0 indegrees for that ids
        for i in range(length):
            if indegrees[i] == 0: #if there is no indegrees
                queue.append(i+1) #i is the index, ids = index + 1
        
        #iterative topological sorting
        self.final_sort = []
        while queue:
            v = queue.pop(0)
            if v not in self.final_sort:
                self.final_sort = self.final_sort + [v]
                queue = mapping[v]+queue
        
        for i in self.final_sort:
            self.hr.append([0,0,self.tasks[i-1]]) #self.task is in order of ids 0,0 can be cahnged to consider time(start, end)
        
        return self.hr
        
    def make_list(self): #complexity analysis = O(n^2)+O(n) = O(n^2)
        #a method that returns self.hr with all the tasks sorted in greedy-optimal order
        self.sorting()
        return self.printout()
    
    def make_new_list(self,new_task): # a method for adding new task in a day, complexity analysis = O(n)
        #new_list should
        #self.tasks.appemd(new_task) -> this way I need to run the sorting again. To consume less time, let's use the already built program and insert our new idea

        if int(len(self.hr))>1:
            #if the tasks are already sorted but new element comes in, insert the new task
            if new_task[3] == []: #if it is not dependent
                for i in range(int(len(self.hr))):
                    if self.hr[i][1][5] > new_task[5]: #O(n) complexity, however, as there are not much tasks per day (humans don't have millions of things to do in a day)
                        #in expense of time compared to having another storage of values is fine as the time has not much difference (when the elements are in small size)
                        self.hr.append([new_task[6], new_task]) #this will change the self.hr
                        break
                        
            else: #when the new element is a dependent one
                place = len(self.final_sort)
                for i in new_task[3]: # find where new_task will belong
                    if place >= self.final_sort.index(i):
                        place = self.final_sort.index(i)
                place += 1 #as self.hr has 1 more element(wake_up)
                self.hr = self.hr[:place] + [[0,0,new_task]] + self.hr[place:]                    
        
        return self.printout() #return the new list of to-do's for the day
        
    def printout(self): #complexity analysis = O(n)
        for i in self.hr:
            #print what we want to see
            if i[2][4] == 1:
                multiability = "multitask-able task"
            else:
                multiability = "" #if it is not multitask-able work, there is no need to say anything
            print(i[2][1] + " [duration = " + str(i[2][2]) + " minutes planned] " + multiability + "\n")
            

In [113]:
#[example test 1] : Test if the scheduler works to make a list of things to do (time complexity = O(n^2))
#ids, 'description', duration, dependencies, multi-tasking, urgency, importance
assignment = todo(1,'assignment', 600, [] , 0, 4, 4)
orchestra = todo(2, 'practice for the orchestra', 60,[],1, 11, 2)
read = todo(3, 'read a book: Clean Coding', 120,[] ,0, -1, 3)
preparation = todo(4, 'prepare for tomorrow\'s field trip lunch box', 30, [1,2],1, -1, 4)
review = todo(5, 'review the classes and memo the points to take with me', 90, [3,4], 0,-1, 3)

#always, tasks should be in the ascending order of ids
tasks = [assignment.aslist(), orchestra.aslist() , read.aslist(), preparation.aslist(), review.aslist()]

#[example test 2]: Test if the scheduler works to make a list out of existing schedule. (time complexity = O(n))
Monday = todo_list(tasks, 6.5) #what to do on monday after waking up at 6:30 am
Monday.make_list()

shower = todo(6, 'take a shower', 20, [1], 0, -1,2)

Monday.make_new_list(shower.aslist())

wakeup [duration = 0 minutes planned] 

assignment [duration = 600 minutes planned] 

prepare for tomorrow's field trip lunch box [duration = 30 minutes planned] multitask-able task

practice for the orchestra [duration = 60 minutes planned] multitask-able task

review the classes and memo the points to take with me [duration = 90 minutes planned] 

read a book: Clean Coding [duration = 120 minutes planned] 

wakeup [duration = 0 minutes planned] 

assignment [duration = 600 minutes planned] 

prepare for tomorrow's field trip lunch box [duration = 30 minutes planned] multitask-able task

practice for the orchestra [duration = 60 minutes planned] multitask-able task

take a shower [duration = 20 minutes planned] 

review the classes and memo the points to take with me [duration = 90 minutes planned] 

read a book: Clean Coding [duration = 120 minutes planned] 



In [18]:
#Topological sorting reference by Guido van Rossum
#https://code.activestate.com/recipes/576723-dfs-and-bfs-graph-traversal/
def recursive_dfs(graph, start, path=[]):
    '''recursive depth first search from start'''
    path=path+[start]
    for node in graph[start]:
        if not node in path:
            path=recursive_dfs(graph, node, path)
    return path

def iterative_dfs(graph, start, path=[]):
    '''iterative depth first search from start'''
    q=[start]
    while q:
        v=q.pop(0)
        if v not in path:
            path=path+[v]
            q=graph[v]+q
    return path

def iterative_bfs(graph, start, path=[]):
    '''iterative breadth first search from start'''
    q=[start]
    while q:
        v=q.pop(0)
        if not v in path:
            path=path+[v]
            q=q+graph[v]
    return path

'''
   +---- A
   |   /   \
   |  B--D--C
   |   \ | /
   +---- E
'''
graph = {'A':['B','C'],'B':['D','E'],'C':['D', 'E'],'D':['E'],'E':['A']}
print('recursive dfs ', recursive_dfs(graph, 'A'))
print('iterative dfs ', iterative_dfs(graph, 'A'))
print('iterative bfs ', iterative_bfs(graph, 'A'))

recursive dfs  ['A', 'B', 'D', 'E', 'C']
iterative dfs  ['A', 'B', 'D', 'E', 'C']
iterative bfs  ['A', 'B', 'C', 'D', 'E']


In [71]:
l = [0]*10
print(l)
i = [0]*len(l)
print(i)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
